In [1]:

import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
# !pip install pygame
from pygame import mixer


pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
eye_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_eye.xml')
model=load_model(r'C:\Users\Hp\Desktop\mrlEyes_2018_01\models\model.h5')


In [14]:
mixer.init()
sound= mixer.Sound(r'C:\Users\Hp\Desktop\mrlEyes_2018_01\alarm.wav')
cap=cv2.VideoCapture(0)
Score=0
while True:
    ret,frame = cap.read()
    height,width=frame.shape[0:2]
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    ##minNeighbors as 3, means it will look for atleast 3 points in the face to detect a face
    faces= face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    ##minNeighbors as 3, means it will look for atleast 4 points in the face to detect eyes like nose, ears, lips 
    eyes=eye_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=4)
    
    cv2.rectangle(frame,(0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),color=(255,0,0),thickness=3)
    
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh),color=(255,0,0),thickness=3)
        eye=frame[ey:ey+eh,ex:ex+w]
        eye=cv2.resize(eye,(80,80))
        eye=eye/255
        eye=eye.reshape(80,80,3)
        eye=np.expand_dims(eye,axis=0)
        
        prediction = model.predict(eye)
        if (prediction[0][0]>0.3):
            cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'Score'+ str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
            ##Increasing the score when eyes are opened
            Score +=1
            if(Score>15):
                try:
                    sound.play()
                except:
                    pass
        ##making ensure to only predict opened eyes when it's probability above 90%    
        elif (prediction[0][1]>0.9):
            cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'Score'+ str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)
            ##Descresing the sccore when eyes are opened and handling it when it is below zero
            Score -=1
            if (Score<0):
                Score=0
    cv2.imshow('frame',frame)
    
    ##To quit from the frame when 'q' is entered 
    if cv2.waitKey(33) & 0XFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 40ms/step


## Results:-

### With Eye Open

#### With-out Eye box
<img src="WhatsApp Image 2022-09-25 at 9.19.01 AM (1).jpeg"  width="500" height="600">

#### With Eye Box
<img src="WhatsApp Image 2022-09-25 at 9.21.06 AM.jpeg"  width="500" height="600">

### With Eye Closed 
<img src="screenshot 2022-09-25 093051.png"  width="500" height="600">

Reference Link :-
https://data-flair.training/blogs/python-project-driver-drowsiness-detection-system/
